# Data Processing: Illusory Pitch

In [ ]:
import numpy as np
import pandas as pd
import scipy.stats as stats
from glob import glob

def dprime_and_c(hit_rate, fa_rate):
    
    # Get corresponding z-scores for the hit rate and false alarm rate
    zH = stats.norm.ppf(hit_rate)
    zF = stats.norm.ppf(fa_rate)
    
    # Calculate d' and C using z-scores
    dprime = zH - zF
    C = -(zH + zF) / 2
    
    return dprime, C

### Load data

In [ ]:
# Find all data files
datafiles = glob('../data/Il*.csv')

# Load each data file and concatenate them into a single table
d = pd.concat((pd.read_csv(f) for f in datafiles))

# Select only non-pilot participants
d = d[d.code_version == 'v1.0']

# Select only trial response events
d = d[d.event == 'response']

### Mark responses as correct or incorrect and save trial data

In [ ]:
# Recode the pitch shift and key press as 0 for "lower" and 1 for "higher"
d = d.assign(answer = d.pitch_shift == '+',
             response = d.key_press.astype(int) == 38)

# Mark whether each response was correct by comparing it to the correct answer
d = d.assign(correct = d.answer == d.response)

# Drop irrelevant columns
d = d.drop(columns=['trial_type', 'trial_index', 'internal_node_id', 'stimulus', 'event', 'button_pressed', 'key_press'])

# Save trial data
d.to_csv('../data/response_data.csv', index=False)

### Calculate scores within each subject and condition

In [ ]:
# Define conditions as (octave, offset) pairs
conditions = [(3, -15), (3, 0), (3, 15), (5, -15), (5, 0), (5, 15)]

# Scores will be stored in a long-format table
scores = pd.DataFrame(columns=['subject', 'octave', 'offset',
                               'hit_rate', 'fa_rate', 'accuracy', 
                               'perc_resp_low', 'dprime', 'C'])

# Calculate scores for each subject
for s, subj in enumerate(d.subject.unique()):
    
    # Select all responses from the current subject
    subj_trials = d[d.subject == subj]
    
    # Calculate scores within each condition
    for i, condition in enumerate(conditions):
        
        # Select all trials from the current condition
        octave = condition[0]
        offset = condition[1]
        trials = subj_trials[(subj_trials.octave == octave) & (subj_trials.offset == offset)]
        
        # Create dictionary to store scores from current subject and condition
        condi_scores = {'subject': subj, 'octave': octave, 'offset': offset}
        
        # Calculate accuracy and the percent of the time the participant responded "lower"
        condi_scores['accuracy'] = np.mean(trials.correct)
        condi_scores['perc_resp_low'] = np.mean(~trials.response)
        
        # Calculate hit and false alarm rates using Hautus (1995) adjustment to avoid 0s and 1s
        condi_scores['hit_rate'] = (np.sum(trials.answer & trials.response) + .5) / (np.sum(trials.answer) + 1)
        condi_scores['fa_rate'] = (np.sum(~trials.answer & trials.response) + .5) / (np.sum(~trials.answer) + 1)
        
        # Calculate d' and C based on the hit rate and false alarm rate
        condi_scores['dprime'], condi_scores['C'] = dprime_and_c(condi_scores['hit_rate'], condi_scores['fa_rate'])
        
        # Add current scores as a row to the full table of scores
        scores.loc[len(scores.index)] = condi_scores

### Save processed scores to a file

In [ ]:
scores.to_csv('../data/scores.csv', index=False)

### Calculate subject-level scores

In [ ]:
offsets = [-15, 0, 15]

# Scores will be stored in a long-format table
scores = pd.DataFrame(columns=['subject', 'dprime', 'C_intercept', 'C_slope'])

# Calculate scores for each subject
for s, subj in enumerate(d.subject.unique()):

    # Create dictionary for holding the current subject's data
    subj_scores = {'subject': subj}
    
    # Select all responses from the current subject
    subj_trials = d[d.subject == subj]
    
    # Calculate hit and false alarm rates using Hautus (1995) adjustment to avoid 0s and 1s
    hit_rate = (np.sum(subj_trials.answer & subj_trials.response) + .5) / (np.sum(subj_trials.answer) + 1)
    fa_rate = (np.sum(~subj_trials.answer & subj_trials.response) + .5) / (np.sum(~subj_trials.answer) + 1)

    # Calculate d' across all of the subject's trials based on the hit rate and false alarm rate
    subj_scores['dprime'], _ = dprime_and_c(hit_rate, fa_rate)
    
    # Calculate C within each offset condition
    C_values = np.zeros(len(offsets), dtype=float)
    for i, offset in enumerate(offsets):
        offset_trials = subj_trials[subj_trials.offset == offset]
        hit_rate = (np.sum(offset_trials.answer & offset_trials.response) + .5) / (np.sum(offset_trials.answer) + 1)
        fa_rate = (np.sum(~offset_trials.answer & offset_trials.response) + .5) / (np.sum(~offset_trials.answer) + 1)
        _, C = dprime_and_c(hit_rate, fa_rate)
        C_values[i] = C

    # Fit a line across the three C values
    slope, intercept = np.polyfit(offsets, C_values, 1)
    
    # Add the slope and intercept of the line to the subject's data dictionary
    subj_scores['C_intercept'] = intercept
    subj_scores['C_slope'] = slope

    # Add the subject's data to the overall dataframe
    scores.loc[len(scores.index)] = subj_scores

In [ ]:
scores.to_csv('../data/subj_scores.csv', index=False)